## Introduction to model stacking using sklearn models

#### This workshop uses House Price Data from Kaggle to demonstrate some frequently used machine learning techniques and a model stacking and blending appraoch

#### Machine Learning Models Covered:
**Sklearn:**   
Extra Trees   
Random Forest   
Lasso Regression   
Ridge Kernel Regression
    
**XGBoost:**   
XGBoost (Extreme Gradient Boosting)   



### Initialization: import required packages   
Import the data sctructure and analysis packages

In [1]:
import pandas as pd
import numpy as np

/Users/meichengshih/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


Import machine learning models

In [2]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Lasso
import xgboost as xgb
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor

Import packages related to model ensembling and the other packages

In [3]:
import time
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import KFold

Read cleaned data, the data cleaning process will not be discussed in this workshop

In [4]:
train=pd.read_csv('./train-b.csv',index_col=0)
to_pred=pd.read_csv('./test-b.csv',index_col=0)
## Read the Log of House Price
target=np.log(pd.read_csv('./target.csv',index_col=0, header=None).astype('float32'))

Define first layer regressors and their corresponding parameters, the parameters were obtained from Bayesian Optimization based parameter tunning process. We will not discuss the details of parameter tuning today. 

This example will contain 5 first-layer regressors

In [5]:
# Define Kernel Rdige Model
kr=KernelRidge(alpha=10**(-1.75),gamma=10**(-2.25),kernel='rbf')

# Define Lasso Model
lasso=Lasso(alpha=10**(-3.65),max_iter=3000)

# Define XGBoost Model
xgbr=xgb.XGBRegressor(colsample_bytree=0.7,gamma=0.02,learning_rate=0.09,max_depth=4, min_child_weight=1,
                      n_estimators=420,reg_alpha=0.1,reg_lambda=0.3,subsample=0.75)

# Define Extra Trees Regressor
extr=ExtraTreesRegressor(max_depth=28,max_features=0.9,n_estimators=450)

# Define Random Forest Regressor
rfr=RandomForestRegressor(max_depth=25,max_features=0.35,n_estimators=400,min_samples_leaf=1)

# Define a list of regressors
frgrs=[kr,lasso,xgbr,extr,rfr] # Kernel Ridge, Lasso, XGBoost, Extra Tress and Random Forest Regressors

Generate performance recorded matrix, and the train and test matrix for the second layer regressor

In [6]:
# Define number of folds used for model stacking
nf=10

# Record the performance of each fold iteration
eval_rec=np.zeros((nf,len(frgrs)))

# Record the train and test matrix for second layer regressor at the end of each fold iteration
blend_train_temp=np.zeros((train.shape[0]))
blend_pred_temp=np.zeros((to_pred.shape[0]))

# Record the final train and test matrix for the second layer regressor
blend_train=np.zeros((train.shape[0],len(frgrs)))
blend_pred=np.zeros((to_pred.shape[0],len(frgrs)))

Start model stacking process

In [7]:
## Record strat time
stime=time.time()
## K-Fold with Shufffle
skf = list(KFold(len(target), nf ,shuffle=True))
## Loop over all regressors
for j, rgr in enumerate(frgrs):
    print (str(j+1)+"th Regressor")
    print ("(Fold,RMSE,Time)")
    ## Loop over all folds
    for i in xrange(nf):
        ### For each fold iteration, determine the train and test data based on the generated fold indices
        trainind, testind=skf[i]
        xtrain, xtest = train.ix[trainind,:], train.ix[testind,:]
        ytrain, ytest = target.ix[trainind], target.ix[testind]
        ### Converting y into 1d array 
        ytrain=ytrain.ix[:,1]
        ### Train a model with the train data x and y
        rgr.fit(xtrain, ytrain)
        ### Predict the y of the test data
        ytest_pred = rgr.predict(xtest).astype('float32')
        ### Record the predicted y of test data
        blend_train_temp[testind]=ytest_pred
        ### Predict the y of the data that needed to be predicted
        pred = rgr.predict(to_pred).astype('float32')
        
        ### Record the y of the data that needed to be predicted    
        if i==0:
            blend_pred_temp=pred
        else:
            blend_pred_temp=blend_pred_temp+pred
        
        ### Show the performance of a model at each fold iteration and the total time spent
        print (i+1, mean_squared_error(ytest,ytest_pred)**0.5, (time.time()-stime))
        ### Record the performance of a model at each fold iteration
        eval_rec[i,j]=mean_squared_error(ytest,ytest_pred)**0.5
    
    ### Record the generated train and predict data from each regressor for the second layer regressor
    blend_train[:,j]=blend_train_temp
    blend_pred[:,j]=blend_pred_temp/float(nf)

1th Regressor
(Fold,RMSE,Time)
(1, 0.11540680043610636, 0.1394059658050537)
(2, 0.15271535813561726, 0.25655102729797363)
(3, 0.13260648681436807, 0.36243295669555664)
(4, 0.11905965242209612, 0.4691450595855713)
(5, 0.1286724093691877, 0.5749070644378662)
(6, 0.13325074762488898, 0.674501895904541)
(7, 0.11743053031285458, 0.7860229015350342)
(8, 0.13924512444218523, 0.8881549835205078)
(9, 0.11017316982513063, 1.000981092453003)
(10, 0.11306101255851288, 1.1066110134124756)
2th Regressor
(Fold,RMSE,Time)
(1, 0.11153387832537427, 1.1369519233703613)
(2, 0.13569516801538153, 1.1691749095916748)
(3, 0.14150482688167307, 1.1972529888153076)
(4, 0.10545438209436676, 1.2318739891052246)
(5, 0.11346430622036149, 1.2671630382537842)
(6, 0.1255775238232511, 1.3090450763702393)
(7, 0.10938045794629248, 1.3440499305725098)
(8, 0.16128093384880088, 1.3902230262756348)
(9, 0.12117078853196341, 1.4256439208984375)
(10, 0.10703711963485971, 1.4575998783111572)
3th Regressor
(Fold,RMSE,Time)
(1, 0.1

Check the average performance of each model under all fold iterations

In [8]:
eval_rec.mean(axis=0)
# 0:Kernel Ridge, 1:Lasso, 2:XGBoost, 3:Extra Tress 4:Random Forest Regressors

array([ 0.12616213,  0.12320994,  0.1249802 ,  0.13853175,  0.13585577])

Set the second layer model

In [9]:
second_rgr=KernelRidge(alpha=10**(-1.35),
            gamma=10**(-1.40),
            kernel='rbf')

Calculate the performance of second layer regressor, the blending regressor

In [10]:
### Generate the performance recording matrix
sec_eval_rec=np.zeros((nf))

for i in xrange(nf):
    ### For each fold iteration, determine the train and test data based on the generated fold indices
    trainind, testind=skf[i]
    xtrain, xtest = blend_train[trainind,:], blend_train[testind,:]
    ytrain, ytest = target.ix[trainind], target.ix[testind]
    ### Converting y into 1d array 
    ytrain=ytrain.ix[:,1]
    ### Train a model with the train data x and y
    second_rgr.fit(xtrain, ytrain)
    ### Predict the y of the test data
    ytest_pred = second_rgr.predict(xtest).astype('float32')
    ### Show the performance of a model at each fold iteration and the total time spent
    print (i+1, mean_squared_error(ytest,ytest_pred)**0.5, (time.time()-stime))
    ### Record the performance of a model at each fold iteration
    sec_eval_rec[i]=mean_squared_error(ytest,ytest_pred)**0.5

(1, 0.10636006887979292, 247.4824459552765)
(2, 0.14727981129726095, 247.54855608940125)
(3, 0.13021683585698793, 247.6120240688324)
(4, 0.10842625802247899, 247.67293000221252)
(5, 0.11846726114705514, 247.73842000961304)
(6, 0.13492956417904833, 247.7994089126587)
(7, 0.10887399094439197, 247.8562150001526)
(8, 0.11466587996493099, 247.91720008850098)
(9, 0.10641949016692044, 247.97866988182068)
(10, 0.10673263252158353, 248.04838109016418)


Check the performance of second layer classifier

In [11]:
sec_eval_rec.mean(axis=0)

0.11823717929804514

#### Compare the performances with a bar chart

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
perf=np.append(eval_rec.mean(axis=0),(sec_eval_rec.mean(axis=0)))
index = np.arange(len(perf))
objects = ('KR', 'Lasso', 'XGB', 'Extra Tress', 'RF', 'Sec_KR')
plt.bar(index, perf, align='center', alpha=0.5)
plt.xticks(index, objects)
plt.ylabel('RMSE')
axes = plt.gca()
axes.set_ylim([0.11, 0.14])
plt.show()

Using Lasso as the second layer classifier

In [ ]:
second_rgr2=Lasso(alpha=10**(-3.35),max_iter=3000)

### Generate the performance recording matrix
lasso_sec_eval_rec=np.zeros((nf))

for i in xrange(nf):
    ### For each fold iteration, determine the train and test data based on the generated fold indices
    trainind, testind=skf[i]
    xtrain, xtest = blend_train[trainind,:], blend_train[testind,:]
    ytrain, ytest = target.ix[trainind], target.ix[testind]
    ### Converting y into 1d array 
    ytrain=ytrain.ix[:,1]
    ### Train a model with the train data x and y
    second_rgr2.fit(xtrain, ytrain)
    ### Predict the y of the test data
    ytest_pred = second_rgr2.predict(xtest).astype('float32')
    ### Show the performance of a model at each fold iteration and the total time spent
    print (i+1, mean_squared_error(ytest,ytest_pred)**0.5, (time.time()-stime))
    ### Record the performance of a model at each fold iteration
    lasso_sec_eval_rec[i]=mean_squared_error(ytest,ytest_pred)**0.5

In [ ]:
lasso_sec_eval_rec.mean(axis=0)

Using XGBoost as the second layer classifier

In [ ]:
second_rgr3=xgb.XGBRegressor(colsample_bytree=0.35,gamma=0.085,learning_rate=0.12,max_depth=5, min_child_weight=5.75,
                      n_estimators=520,reg_alpha=0.6,reg_lambda=4.75,subsample=0.25)

### Generate the performance recording matrix
xgb_sec_eval_rec=np.zeros((nf))

for i in xrange(nf):
    ### For each fold iteration, determine the train and test data based on the generated fold indices
    trainind, testind=skf[i]
    xtrain, xtest = blend_train[trainind,:], blend_train[testind,:]
    ytrain, ytest = target.ix[trainind], target.ix[testind]
    ### Converting y into 1d array 
    ytrain=ytrain.ix[:,1]
    ### Train a model with the train data x and y
    second_rgr3.fit(xtrain, ytrain)
    ### Predict the y of the test data
    ytest_pred = second_rgr3.predict(xtest).astype('float32')
    ### Show the performance of a model at each fold iteration and the total time spent
    print (i+1, mean_squared_error(ytest,ytest_pred)**0.5, (time.time()-stime))
    ### Record the performance of a model at each fold iteration
    xgb_sec_eval_rec[i]=mean_squared_error(ytest,ytest_pred)**0.5

In [ ]:
xgb_sec_eval_rec.mean(axis=0)

#### Comparison between the difference models as the second layer regressor, the red line is the best performance of first layer models

In [ ]:
perf=np.append((sec_eval_rec.mean(axis=0)),(lasso_sec_eval_rec.mean(axis=0)))
perf=np.append(perf,(xgb_sec_eval_rec.mean(axis=0)))
index = np.arange(len(perf))
objects = ('Sec_KR', 'Sec_Lasso', 'Sec_XGB')
plt.bar(index, perf, align='center', alpha=0.5)
plt.xticks(index, objects)
plt.ylabel('RMSE')
axes = plt.gca()
axes.set_ylim([0.11, 0.125])
plt.axhline(y=0.12314992, color='red')
plt.show()

#### The reason why KR performs better than Lasso and XGB as the second layer regressor is because KR can deal with the homogeneious inputs better.  Based on this idea, the use of Neural Network as the second layer  is expected to generate a good quality of output too.